# Selenium Webscraping
## Notebook de clase

En este notebook podras encontrar el ejercicio que hicimos en clase para aprender las bases de selenium

Hagamos los imports relevantes y conectémonos al driver de chrome. Se pueden usar otros navegadores, pero para tener orden usaremos este

In [4]:
from selenium import webdriver

# Por si falla algo. URL al driver de Chrome (descargar el driver desde https://sites.google.com/chromium.org/driver/)
chrome_driver_path = 'driver/chromedriver_mac64/chromedriver' #remplazar esto con el path al chrome driver de tu compu

options = webdriver.ChromeOptions()

driver = webdriver.Chrome(options=options)

El *driver* es el objeto que nos va a permitir hacer la automatización con Selenium. 

No es necesario que se abra Google Chrome:

In [5]:
driver.close()

Podemos trabajar sin necesidad de la interfaz gráfica. Esto es muy útil una vez que hemos terminado de debuggear nuestro código (ya estamos obteniendo los datos que queremos) pues consume menos RAM y es más rápido

In [6]:
options.add_argument('--headless')  # Ejecutar en modo headless (sin interfaz gráfica)

driver = webdriver.Chrome(options=options)

Utilizaremos un sitio web de aprendizaje. Nota: en la vida diaria los sitios se van actualizando, lo cual puede dificultar mantener el código al día y funcionando 

In [7]:
options = webdriver.ChromeOptions()

driver = webdriver.Chrome(options=options)

In [13]:
url = 'https://scrapepark.org/courses/spanish/'

driver.get(url)

Obtengamos algunas cosas básicas

In [14]:
#este import puede o no ser necesario
from selenium.webdriver.common.by import By

In [16]:
nom_pagina = driver.title #titulo de la página
print(nom_pagina)

ScrapePark.org


In [23]:
#Obtengamos la información que se encuentra en el footer de la página
info_marca = driver.find_element(By.CLASS_NAME, 'information-f') # buequda por clase
print(info_marca.text)

testimonios = driver.find_elements_by_class_name('detail-box') # buequda por clase
for testimonio in testimonios:
    test = testimonio.find_element(By.TAG_NAME, 'p')
print(test.text) # .text nos da el texto de un elemento


DIRECCIÓN: Calle falsa 123
TELÉFONO: 4-444-4444
EMAIL: ventas@mail.com


TypeError: 'WebElement' object is not iterable

¿qué pasa si queremos el testimonio de todos los clientes? Necesitamos interectuar con el sitio a través de pithon

In [ ]:

testimonio = driver.find_element(By.ID, 'testimonios')
print("El texto es:", testimonio.text) # .text nos da el texto de un elemento
print('\n')

next = driver.find_element(By.CLASS_NAME, 'carousel-control-next')
next.click()

testimonio = driver.find_element(By.ID, 'testimonios')
print("El texto es:", testimonio.text) # .text nos da el texto de un elemento



## paciencia
en este caso se tiene que tomar en cuenta la velocidad del click contra la velocidad de extraccion y definición de un elemento

In [ ]:
import time

testimonio = driver.find_element(By.ID, 'testimonios')
print(testimonio.text)
print('\n')

next = driver.find_element(By.CLASS_NAME, 'carousel-control-next')
next.click()

time.sleep(.5) #espera medio segundo

testimonio = driver.find_element(By.ID, 'testimonios')
print(testimonio.text) 


¿Cómo localizar un objeto usando su Xpath.

Ventaja: efectividad; 
Desventaja: trabajo

Xpath Syntax:
//tagName[@AttributeName="Value"]

In [ ]:
#//a[@href= "#productos"]
botonProductos= driver.find_element(By.XPATH,'//a[@href= "#productos"]')
#botonProductos= driver.find(By.XPATH('//*[@id="navbarSupportedContent"]/ul/li[3]/a'))# Ej. No funciona
botonProductos.click()


In [ ]:
detailBoxes= driver.find_elements(By.XPATH,'//div[@class= "detail-box"]') #detail boxes es una lista

for box in detailBoxes:
    print(box.text)


In [ ]:
#Queremos crear un data frame de los productos con sus precios 
#usaremos regex para obtener solo el texto de las boxes que necesitamos
import re

patronProducto= r"Patineta (Nueva|Usada) \d{1,2}"

#arreglo de productos
matches = [box for box in detailBoxes if re.search(patronProducto, box.text)]

for box in matches:
    print(box.text)


In [ ]:
import pandas as pd

pattern = r"Patineta (Nueva|Usada) (\d{1,2})\s*\n*\s*\$(\d+)" #ahora más específico

# Crear listas para almacenar los productos, estados, números y precios
productos = [] #un producto es el estado de la patineta con su número
estados = []
numeros = []
precios = []

# Iteramos
for match in matches:
    resultado = re.search(pattern, match.text)
    if resultado:
        #separamos y definimos variables
        tipo = resultado.group(1) + (resultado.group(2))
        estado = resultado.group(1) 
        numero = int(resultado.group(2))
        precio = int(resultado.group(3))
        
        #agregamos a las listas
        productos.append(tipo)
        estados.append(estado)
        numeros.append(numero)
        precios.append(precio)

# Crear el DataFrame usando pandas
data = {
    'Producto': productos,
    'Estado': estados,
    'Número': numeros,
    'Precio': precios
}

df = pd.DataFrame(data)

print(df)

Ya treniendo el data frame podemos usar las funciones de pandas para analizar los datos

In [ ]:
print('Precio promedio:',df['Precio'].mean())
print('Precio mínimo:',df['Precio'].min())
print('Precio máximo',df['Precio'].max())

In [ ]:
print('Precio promedio de las patinetas usadas', df[df['Estado'] == 'Usada']['Precio'].mean())
print('Precio promedio de las patinetas nuevas', df[df['Estado'] == 'Nueva']['Precio'].mean())


print('Rango de precios de las patinetas usadas', df[df['Estado'] == 'Usada']['Precio'].max()-df[df['Estado'] == 'Usada']['Precio'].min())
print('Rango de precios de las patinetas nuevas', df[df['Estado'] == 'Nueva']['Precio'].max()-df[df['Estado'] == 'Nueva']['Precio'].min())

In [ ]:
# Cerrar el navegador
#driver.quit()

